#**IMDB Review Analysis**
## **Parameshwari - CB.SC.I5DAS18026**

### 1. Importing all neccessary libraries

In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk import pos_tag
import re
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, accuracy_score
import gensim
from gensim.parsing.preprocessing import remove_stopwords
!pip install num2words
from num2words import num2words
import sys
import os
py_file_location = "/content/drive/My Drive/NLP"
sys.path.append(os.path.abspath(py_file_location))
from contraction import CONTRACTION_MAP
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### 2. Loading the dataset

In [ ]:
df = pd.read_csv("/content/drive/My Drive/NLP/IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.sentiment.value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

### 3. Text-preprocessing

In [ ]:
def expand_contractions(contraction):
  match = contraction.group(0)
  first_char = match[0]
  if contraction_mapping.get(match):
    expanded_contraction = contraction_mapping.get(match)
  else:
    expanded_contraction = contraction_mapping.get(match.lower())
    expanded_contraction = first_char+expanded_contraction[1:]
  return expanded_contraction

#removing contractions
clean_text = []
contraction_mapping = CONTRACTION_MAP
contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
flags=re.IGNORECASE|re.DOTALL)
for i in range(len(df)):
  text = df['review'][i].lower()
  expanded_text = contractions_pattern.sub(expand_contractions, text)
  clean_text.append(re.sub("'", "", expanded_text))
df['clean_review'] = clean_text

In [ ]:
lemma = WordNetLemmatizer()
def clean(text):
  text = re.sub(pattern = r'<.*?>', repl = '', string = text)
  text = re.sub(pattern = r'\s[a-zA-Z]\s', repl = '', string = text)
  text = re.sub(r'[^a-zA-Z0-9\s]+', '', text)
  text = remove_stopwords(text)
  after_split = text.split()
  for idx in range(len(after_split)):
    if after_split[idx].isdigit():
      after_split[idx] = num2words(after_split[idx])
    text = ' '.join(after_split)
  lemm_word = []
  for token,tag in pos_tag(word_tokenize(text)):
    pos = tag[0].lower()
    if pos not in ['a', 'r', 'n', 'v']:
      pos = 'n'
    lemm_word.append(lemma.lemmatize(token, pos))
    text = ' '.join(lemm_word)
  return text 

df['clean_review'] = df['clean_review'].apply(clean)
df.head()

,review,sentiment,clean_review
0,One of the other reviewers has mentioned that ...,positive,reviewer mention watch one oz episode hook rig...
1,A wonderful little production. <br /><br />The...,positive,wonderful little production film technique una...
2,I thought this was a wonderful way to spend ti...,positive,think waswonderful way spend time ontoo hot su...
3,Basically there's a family where a little boy ...,negative,basically isfamily wherelittle boy jake think ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter matteis love time money isvisually stun...


In [ ]:
from google.colab import files
df.to_csv('df_preprocessed.csv') 
files.download('df_preprocessed.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/My Drive/NLP/df_preprocessed.csv", usecols = ['clean_review', 'sentiment'])
df.head()

,sentiment,clean_review
0,positive,reviewer mention watch one oz episode hook rig...
1,positive,wonderful little production film technique una...
2,positive,think waswonderful way spend time ontoo hot su...
3,negative,basically isfamily wherelittle boy jake think ...
4,positive,petter matteis love time money isvisually stun...


### 4. Text Vectorization using Bow and tf-idf

In [ ]:
cv = CountVectorizer(max_features = 2500)
x_bow = cv.fit_transform(df['clean_review']).toarray()
y = df['sentiment']

In [ ]:
tfidf = TfidfVectorizer()
x_tfidf = tfidf.fit_transform(df['clean_review']).toarray()

### 5. Label Encoding

In [ ]:
le = LabelEncoder()
y = le.fit_transform(y)
y

### 6. Train-test split

In [ ]:
x_train_bow, x_test_bow, y_train_bow, y_test_bow = train_test_split(x_bow, y, test_size = 0.2, random_state = 1)

In [ ]:
x_train_tfidf, x_test_tfidf, y_train_tfidf, y_test_tfidf = train_test_split(x_tfidf, y, test_size = 0.2, random_state = 1)

### 7. Model Evaluation


> a) Logistic Regression

In [ ]:
lr = LogisticRegression()
lr.fit(x_train_bow, y_train_bow)
lr_y_pred_bow = lr.predict(x_test_bow)
print('Accuracy:', accuracy_score(lr_y_pred_bow, y_test_bow))
print("F1 Score: ", f1_score(lr_y_pred_bow, y_test_bow))

Accuracy: 0.8597
F1 Score:  0.8605783563549637


In [ ]:
lr = LogisticRegression()
lr.fit(x_train_tfidf, y_train_tfidf)
lr_y_pred = lr.predict(x_test_tfidf)
print('Accuracy:', accuracy_score(lr_y_pred_tfidf, y_test_tfidf))
print("F1 Score: ", f1_score(lr_y_pred_tfidf, y_test_tfidf))

> b) Naive-bayes

In [ ]:
nb = MultinomialNB()
nb.fit(x_train_bow, y_train_bow)
nb_y_pred_bow = nb.predict(x_test_bow)
print('Accuracy:', accuracy_score(nb_y_pred_bow, y_test_bow))
print("F1 Score: ", f1_score(nb_y_pred_bow, y_test_bow))

Accuracy: 0.8302
F1 Score:  0.8291062801932367


In [ ]:
nb = MultinomialNB()
nb.fit(x_train_tfidf, y_train_tfidf)
nb_y_pred_tfidf = nb.predict(x_test_tfidf)
print('Accuracy:', accuracy_score(nb_y_pred_tfidf, y_test_tfidf))
print("F1 Score: ", f1_score(nb_y_pred_tfidf, y_test_tfidf))



> c) Linear SVM



In [ ]:
svc = LinearSVC()
svc.fit(x_train_bow,y_train_bow)
svc_y_pred_bow = svc.predict(x_test_bow)
print('Accuracy:', accuracy_score(svc_y_pred_bow, y_test_bow))
print("F1 Score: ", f1_score(svc_y_pred_bow, y_test_bow))

Accuracy: 0.8571
F1 Score:  0.8587525946426807


In [ ]:
svc = LinearSVC()
svc.fit(x_train_tfidf,y_train_tfidf)
svc_y_pred_tfidf = svc.predict(x_test_tfidf)
print('Accuracy:', accuracy_score(svc_y_pred_tfidf, y_test_tfidf))
print("F1 Score: ", f1_score(svc_y_pred_tfidf, y_test_tfidf))



> d) Stochastic gradient descent



In [ ]:
sgd = SGDClassifier()
sgd.fit(x_train_bow,y_train_bow)
sgd_y_pred_bow = sgd.predict(x_test_bow)
print('Accuracy:', accuracy_score(sgd_y_pred_bow, y_test_bow))
print("F1 Score: ", f1_score(sgd_y_pred_bow, y_test_bow))

Accuracy: 0.8567
F1 Score:  0.858664562580136


In [ ]:
sgd = SGDClassifier()
sgd.fit(x_train_tfidf,y_train_tfidf)
sgd_y_pred_tfidf = sgd.predict(x_test_tfidf)
print('Accuracy:', accuracy_score(sgd_y_pred_tfidf, y_test_tfidf))
print("F1 Score: ", f1_score(sgd_y_pred_tfidf, y_test_tfidf))